# (one-step) Biot equation - NOTE: there might be a mistake somewhere

In this tutorial we present how to solve the (static) Biot equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unknown is the displacement $u$, the flux $q$ and the pressure $p$.

Let $\Omega$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$\lambda$ Lamé constant and $\mu$ the Kirchhoff modulus, $K$ the permeability of the porous medium and $s_0$ its storativity, and $\alpha$ the Biot-Willis coefficient fot the coupling between the two models.
We want to solve the following problem: find $({u}, p)$ such that
$$
    \begin{aligned}
         & \nabla \cdot [2 \mu {\epsilon}({u}) + \lambda \nabla \cdot u I
         - \alpha p {I}] = -{b}   \\
         & \partial_t (s_0 p + \alpha \nabla \cdot { u}) +
        \nabla \cdot \left( \mu^{-1} K \nabla p\right) = \psi
    \end{aligned}
    \quad \text{in } \Omega.
$$
with $\epsilon$ the symmetric gradient and $b$ a body force. 

In particular, we consider only the spatial structure of the previous problem by doing only one time step. We call the latter $\Delta t$ and write the discrete problem as
$$
    \begin{bmatrix}
        K &  -\alpha D^\top \\
        \alpha D & s_0 M + \Delta t A
    \end{bmatrix}
    \begin{bmatrix}
        u \\ p
    \end{bmatrix}
    =
    \begin{bmatrix}
        b \\ \Delta t \psi
    \end{bmatrix}
$$
where $K$ is the stiffness matrix associated with the elastic problem, $D$ is the coupling between the two physics, $M$ is the mass matrix associated with the pressure and $A$ is the stiffness matrix associated to the diffusive part of the pressure. All the aforementioned matrices are properly scaled by their physical parameters if not explicitly written. The second row has been multiply by $\Delta t$ to preserve the skew-symmetry of the problem.

## Exercise 2: fixed-stress iterative solver

To solve the discrete problem we employ the fixed-stress iterative solver which reads:
given $\beta > 0$ and $(u^i, p^i)$ compute $(u^{i+1}, p^{i+1})$ by solving until convergence
$$
[(s_0 +\beta) M + \Delta t A] p^{i+1} = \Delta t \psi- \alpha D u^{i} + \beta M p^i
$$

followed by
$$
K u^{i+1} = b + \alpha D^\top p^{i+1}
$$

Note that i) we are performing only one time step starting from $u=0$, $p=0$, therefore the right hand side is simplified and ii) the two system are equivalent to the following block triangular one:

$$
    \begin{bmatrix}
        K & -\alpha D^\top \\
        0 & (s_0 + \beta) M + \Delta t A
    \end{bmatrix}
    \begin{bmatrix}
        u^{i+1} \\ p^{i+1}
    \end{bmatrix}
    =
    \begin{bmatrix}
        0 & 0 \\
        -\alpha D& \beta M
    \end{bmatrix}
    \begin{bmatrix}
        u^{i} \\ p^{i}
    \end{bmatrix}
    +
    \begin{bmatrix}
        b \\ \Delta t \psi
    \end{bmatrix}
$$


A footing problem is when a force is impose on the top compressing the body and the bottom is fixed. In this case we assume that the force is applied on the central portion of the top boundary.

For this test case we set $\Omega = [0, 3] \times [0, 1]$, $b = 0$, and the following boundary conditions:
$$ 
\begin{aligned}
    &u = 0 &&\text{ and }&& \nu \cdot q = 0 &&\text{ on } \partial \Omega  \setminus \partial_{top} \Omega
\\
&\nu \cdot \sigma = 0 &&\text{ and }&& p = 0 &&\text{ on } \partial_{top} \Omega \setminus \partial_s \Omega
\\
&\nu \cdot \sigma = [0, -1]^\top &&\text{ and }&& p = 0 &&\text{ on }\partial_s \Omega 
\end{aligned}
$$
where $\partial_{s} \Omega = [1, 2] \times \{1\}$.



We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [101]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg


# useful for the stopping criteria in the iterative solvers
def compute_err(x, x_i, mass):
    diff = np.sqrt((x_i - x) @ mass @ (x_i - x))
    norm = np.sqrt(x_i @ mass @ x_i)
    return diff / norm if norm else diff

We create now the grid, since we use a vector Lagrangian of order 1 for ${u}$ and Raivar-Thomas for the flux $q$, we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 3d.

In [102]:
mesh_size = 0.05
delta_t = 0.01
dim = 2

pts = np.array(
    [
        [0, 3, 3, 2, 1, 0],
        [0, 0, 1, 1, 1, 1],
    ]
)
sd = pg.grid_from_boundary_pts(pts, mesh_size, as_mdg=False)
sd.compute_geometry()

# NOTE: to be compliant with PorePy sparse matrices
sd.cell_faces = sps.csc_matrix(sd.cell_faces)

# data for the iterative solvers
tol = 1e-4
max_iter = 1e2

With the following code we set the data and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [103]:
# the physical parameters of the problem, assumed constant
alpha = 1
s0 = 0.5  # 0.1 and 1

E = 1
nu = 0.25
fun = -1e-3

# physical Lamé parameters (keep separate from per-cell arrays)
lambda_phys = E * nu / ((1 + nu) * (1 - 2 * nu))
mu_phys = E / (2 * (1 + nu))

Let us set up the flow data.

In [104]:
key_p = "flow"

# Permeability
perm = pp.SecondOrderTensor(np.ones(sd.num_cells))

# Boundary conditions
b_faces = sd.tags["domain_boundary_faces"].nonzero()[0]

# With the following steps we identify the portions of the boundary
# to impose the boundary conditions
top = np.logical_and.reduce(
    (
        np.isclose(sd.face_centers[1, b_faces], 1),
        sd.face_centers[0, b_faces] > 1,
        sd.face_centers[0, b_faces] < 2,
    )
)

# Set the labels for the boundary conditions
labels = np.array(["neu"] * b_faces.size)
labels[top] = "dir"

bc = pp.BoundaryCondition(sd, b_faces, labels)
bc_val = np.zeros(sd.num_faces)

# source term
source = np.zeros(sd.num_cells)

# collect all data
data_p = {
    pp.PARAMETERS: {
        key_p: {
            "second_order_tensor": perm,
            "bc_values": bc_val,
            "bc": bc,
            "source": source,
        }
    },
    pp.DISCRETIZATION_MATRICES: {key_p: {}},
}

Let us declare data associated to the elastic problem.

In [105]:
key_u = "mechanics"

# Create stiffness matrix
lambda_ = lambda_phys * np.ones(sd.num_cells)
mu = mu_phys * np.ones(sd.num_cells) / 2  # MPSA scaling retained
C = pp.FourthOrderTensor(mu, lambda_)

# Define boundary type
b_faces = sd.get_all_boundary_faces()
num_b_faces = b_faces.size
labels = np.array(["neu"] * num_b_faces)

bottom = np.isclose(sd.face_centers[1, b_faces], 0)
labels[bottom] = "dir"
bc = pp.BoundaryConditionVectorial(sd, b_faces, labels)

bc_val = np.zeros((sd.dim, sd.num_faces))

top = np.logical_and.reduce(
    (
        np.isclose(sd.face_centers[1, :], 1),
        sd.face_centers[0, :] > 1,
        sd.face_centers[0, :] < 2,
    )
)
bc_val[1, top] = fun * sd.face_areas[top]
bc_val = bc_val.ravel("F")

# define source term
source = np.zeros((sd.dim, sd.num_cells)).ravel("F")

# collect all data
data_u = {
    pp.PARAMETERS: {
        key_u: {
            "fourth_order_tensor": C,
            "bc_values": bc_val,
            "bc": bc,
            "source": source,
            "scalar_vector_mappings": {key_u: 0.5},
        }
    },
    pp.DISCRETIZATION_MATRICES: {key_u: {}},
}

Once the data are assigned to the grid, we construct the matrices. Once the latter is created, we also construct the right-hand side containing the boundary conditions.

In [106]:
# discretization for the mechanical part
disc_u = pp.Mpsa(key_u)
disc_u.discretize(sd, data_u)

# discretization for the flow part
disc_p = pp.Mpfa(key_p)
disc_p.discretize(sd, data_p)

# discretization for the coupling part
disc_coupling = pp.Biot(key_u)
disc_coupling.discretize(sd, data_u)

dof_u, dof_p = sd.num_cells * sd.dim, sd.num_cells
dofs = np.array([dof_u, dof_p])

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (displacement boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom.

In [118]:
# compute storage coefficient beta and drained bulk modulus K_dr
K_dr = lambda_phys + 2 * mu_phys / dim
beta = alpha**2 / K_dr

# definition of the discretization matrices
K, rhs_u = disc_u.assemble_matrix_rhs(sd, data_u)
A, rhs_p = disc_p.assemble_matrix_rhs(sd, data_p)
M = sps.diags(sd.cell_volumes)

# the coupling term
D = data_u[pp.DISCRETIZATION_MATRICES][key_u]["displacement_divergence"][key_u]

sp_fluid = (s0 + beta) * M + delta_t * A

step = 0
err = tol + 1

# initialization of the solution
u_i = np.zeros(dof_u)
p_i = np.zeros(dof_p)

while err > tol and step < max_iter:
    # for a given u solve the flow problem
    rhs_fluid = delta_t * rhs_p - alpha * D @ u_i + beta * M @ p_i

    ls = pg.LinearSystem(sp_fluid, rhs_fluid)
    p = ls.solve()

    # for a given p solve the mech problem
    rhs_mech = rhs_u + alpha * D.T @ p

    ls = pg.LinearSystem(K, rhs_mech)
    u = ls.solve()

    # compute the stopping criteria
    step += 1

    u_i_dim = np.reshape(u_i, (sd.dim, -1), order="F")
    u_dim = np.reshape(u, (sd.dim, -1), order="F")
    err_u = np.sum([compute_err(u_i_d, u_d, M) for u_i_d, u_d in zip(u_i_dim, u_dim)])
    err_p = compute_err(p_i, p, M)
    err = err_u + err_p

    print(step, err, err_u, err_p)

    # save for the next non-linear iteration step
    u_i = u.copy()
    p_i = p.copy()

1 2.0 2.0 0.0
2 1.3530184190830679 0.35301841908306786 1.0
3 0.8155456628490396 0.34758626848573093 0.46795939436330874
4 0.6347972525320947 0.34134541332481616 0.2934518392072786
3 0.8155456628490396 0.34758626848573093 0.46795939436330874
4 0.6347972525320947 0.34134541332481616 0.2934518392072786
5 0.5330750178407929 0.3257464072708203 0.20732861056997262
6 0.4535862353083894 0.29727271639489355 0.15631351891349585
7 0.38279765554919043 0.2600325457811832 0.12276510976800727
5 0.5330750178407929 0.3257464072708203 0.20732861056997262
6 0.4535862353083894 0.29727271639489355 0.15631351891349585
7 0.38279765554919043 0.2600325457811832 0.12276510976800727
8 0.3205502825644477 0.22139251770515192 0.09915776485929577
9 0.26814874046922577 0.18640751779266534 0.0817412226765604
10 0.22526308332388353 0.15682603557997962 0.06843704774390391
8 0.3205502825644477 0.22139251770515192 0.09915776485929577
9 0.26814874046922577 0.18640751779266534 0.0817412226765604
10 0.22526308332388353 0.156

Since the computed $u$ is a vector per peak of the grid, for visualization purposes we project the displacement in each cell center as vector. Similarly for the flow field $q$. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [108]:
# reshape the displacement for the export
u_3d = np.reshape(u, (sd.dim, -1), order="F")
if dim == 2:
    u_3d = np.vstack((u_3d, np.zeros(sd.num_cells)))

# export the final solution
save = pp.Exporter(sd, "sol", folder_name="ex1")
save.write_vtu(
    [
        ("cell_p", p),
        ("cell_u", u_3d),
    ],
)

In [ ]:
# Consistency check
# assert np.isclose(np.linalg.norm(p), 0.06432719631010395)
# assert np.isclose(np.linalg.norm(u), 0.04120435804000858)